In [1]:
import torch
import torch.nn as nn
from torch.utils.data import Dataset
from torch.utils.data import DataLoader
import torch.optim as optim
import numpy as np
import matplotlib.pyplot as plt

import pickle as pkl
from pathlib import Path
from IPython.display import clear_output
import ipywidgets as widgets
from time import sleep

from FNO import FNO

In [2]:
# global parameters
B = 5 # batch size
nx = 500 # npts
Nsols = 500

modes = 10
num_fourier_layers = 4
in_channels = 4
out_channels = 3
fourier_channels = 20
activation = nn.GELU()

epochs = 1
visualize = True

In [3]:
with open("./outputs_train.pkl", 'rb') as file:
    output_data = pkl.load(file)
len(output_data)

375

# Initialize Data Loader

In [23]:
class SodShockData(Dataset):
    def __init__(self, path, split: str = "train", normalize: bool = True):
        super(SodShockData, self).__init__()
        with open(Path(path) / f"inputs_{split}.pkl", 'rb') as file:
            self.input_data = pkl.load(file)

        with open(Path(path) / f"outputs_{split}.pkl", 'rb') as file:
            self.output_data = pkl.load(file)
        self.normalize = normalize
    def __len__(self):
        return len(self.input_data)
    
    def __getitem__(self, idx:int):
        inp = self.input_data[idx]
        x, rho, u, p = inp
        input_data = torch.stack((torch.as_tensor(x),torch.as_tensor(rho),
                                  torch.as_tensor(u),torch.as_tensor(p)), dim=-1)

        out = self.output_data[idx]
        rho, u, p = out
        output_data = torch.stack((torch.as_tensor(rho),torch.as_tensor(u),
                                   torch.as_tensor(p)), dim=-1)
        
        if self.normalize:
            input_mean = input_data.mean()
            input_std = input_data.std()
            input_data = (input_data-input_mean)/input_std
            
            output_mean = output_data.mean()
            output_std = output_data.std()
            output_data = (output_data-output_mean)/output_std
        
        return (input_data, output_data)

In [24]:
train_dataset = SodShockData(
    path="./",
    split="train",
)

train_loader = DataLoader(
    train_dataset,
    batch_size=B,
    shuffle=True,
)
len(train_dataset)

375

In [26]:
for batch in train_loader:
    inputs, outputs = batch
    print(outputs)
    break

tensor([[[ 2.3926, -0.0989, -0.9345],
         [ 2.3926, -0.0989, -0.9345],
         [ 2.3926, -0.0989, -0.9345],
         ...,
         [-0.8509, -0.8300, -0.9345],
         [-0.8509, -0.8300, -0.9345],
         [-0.8509, -0.8300, -0.9345]],

        [[ 2.3536, -0.0492, -0.9387],
         [ 2.3536, -0.0492, -0.9387],
         [ 2.3536, -0.0492, -0.9387],
         ...,
         [-0.8497, -0.8275, -0.9387],
         [-0.8497, -0.8275, -0.9387],
         [-0.8497, -0.8275, -0.9387]],

        [[ 1.6837,  0.9367, -1.0461],
         [ 1.6837,  0.9367, -1.0461],
         [ 1.6837,  0.9367, -1.0461],
         ...,
         [-0.8478, -0.7982, -1.0461],
         [-0.8478, -0.7982, -1.0461],
         [-0.8478, -0.7982, -1.0461]],

        [[ 2.2698,  0.0633, -0.9524],
         [ 2.2698,  0.0633, -0.9524],
         [ 2.2698,  0.0633, -0.9524],
         ...,
         [-0.8508, -0.8255, -0.9524],
         [-0.8508, -0.8255, -0.9524],
         [-0.8508, -0.8255, -0.9524]],

        [[ 1.9337,  0.57

In [ ]:
# model initialization
model = FNO(modes=modes, num_fourier_layers=num_fourier_layers, in_channels=in_channels, fourier_channels=fourier_channels, out_channels=out_channels, activation=activation)

# initialize optimizer
lr = 0.001
optimizer = optim.Adam(model.parameters(), lr=lr)
l = nn.MSELoss(reduction='None')
epochs = 100
for epoch in range(epochs):
    for batch in train_loader:
        inputs, outputs = batch
        inputs = inputs.float()
        outputs = outputs.float()
        #print(inputs.shape)
        #print(outputs.shape)
        pred = model(inputs)
        #print(pred.shape)
        
        optimizer.zero_grad()
        loss = l(pred, outputs)
        loss.backward(retain_graph=True)
        optimizer.step()
        # print(loss)
        #zero_grad
        #backwards
        #step
        # print(inputs.shape, outputs.shape)
    if epoch % 20 == 0:
        print(f'Epoch [{epoch+1}/{epochs}], Loss: {loss}')


Epoch [1/8000], Loss: 0.44120484590530396
Epoch [21/8000], Loss: 0.21578438580036163
Epoch [41/8000], Loss: 0.2466602325439453


KeyboardInterrupt: 

In [11]:
x = np.zeros(100)
rho = np.zeros(100)
#torch.stack((torch.as_tensor(x), torch.as_tensor(rho)), dim=-1)

In [ ]:
def activation(name='tanh'):
    if name in ['tanh', 'Tanh']:
        return nn.Tanh()
    elif name in ['relu', 'ReLU']:
        return nn.ReLU(inplace=True)
    elif name in ['lrelu', 'LReLU']:
        return nn.LeakyReLU(inplace=True)
    elif name in ['sigmoid', 'Sigmoid']:
        return nn.Sigmoid()
    elif name in ['softplus', 'Softplus']:
        return nn.Softplus(beta=4)
    elif name in ['celu', 'CeLU']:
        return nn.CELU()
    elif name in ['elu']:
        return nn.ELU()
    elif name in ['SiLU']:
        return nn.SiLU()
    else:
        raise ValueError('Unknown activation function')

TypeError: 'int' object is not iterable

In [ ]:
# dataloader intialization

with open("sod_shock_dataset.pkl", "rb") as f:
    solutions = pkl.load(f)


# extract data

# train dataset

# test dataset

In [12]:
for (init_cond, positions, regions, values) in solutions:
    x = values['x']
    rho = values['rho']
    u = values['u']
    p = values['p']
    break
rho.shape
x

array([0.        , 0.00200401, 0.00400802, 0.00601202, 0.00801603,
       0.01002004, 0.01202405, 0.01402806, 0.01603206, 0.01803607,
       0.02004008, 0.02204409, 0.0240481 , 0.0260521 , 0.02805611,
       0.03006012, 0.03206413, 0.03406814, 0.03607214, 0.03807615,
       0.04008016, 0.04208417, 0.04408818, 0.04609218, 0.04809619,
       0.0501002 , 0.05210421, 0.05410822, 0.05611222, 0.05811623,
       0.06012024, 0.06212425, 0.06412826, 0.06613226, 0.06813627,
       0.07014028, 0.07214429, 0.0741483 , 0.0761523 , 0.07815631,
       0.08016032, 0.08216433, 0.08416834, 0.08617234, 0.08817635,
       0.09018036, 0.09218437, 0.09418838, 0.09619238, 0.09819639,
       0.1002004 , 0.10220441, 0.10420842, 0.10621242, 0.10821643,
       0.11022044, 0.11222445, 0.11422846, 0.11623246, 0.11823647,
       0.12024048, 0.12224449, 0.1242485 , 0.12625251, 0.12825651,
       0.13026052, 0.13226453, 0.13426854, 0.13627255, 0.13827655,
       0.14028056, 0.14228457, 0.14428858, 0.14629259, 0.14829

In [29]:
# optimizer

In [30]:
# loss function

In [31]:
# training loop

# 